In [2]:
#Importing all the tools
#API_extraction
import os
from dotenv import load_dotenv
import json
import requests
load_dotenv()
from IPython.display import display, Image
import re
import math
import pymongo
from pymongo import MongoClient
import itertools


In [ ]:
#Extracting pull request info from API(Not sure how to navigate through pages)
#All of them or only the closed ones??????

def requestGitHub (n=1, token=os.getenv("apikey"),queryParams={}):
    
    baseUrl='https://api.github.com'
    endpoint = f'/repos/ironhack-datalabs/datamad0820/pulls?page={n}&q=state%3Aall'

    url = baseUrl+endpoint

    headers = {
        "Authorization": f"token {token}"}

    res = requests.get(url, headers=headers)
    
    response = res.json()
    
    if res.status_code != 200:
        raise ValueError(f'Invalid github api call: {data["message"]}')
    
    return response


In [ ]:
data = requestGitHub()

In [ ]:
'''data[0]'''

In [ ]:
'''display(data[0]['user'])
''''''display(data[0]['body'])'''

In [ ]:
x=11

print(re.search(r'[lab-].*\]',data[x]['title']).group().split(']')[0])

print(data[x]['user']['login'])

print(re.search(r"\] ([A-Za-z0-9-]+ [A-Za-z0-9-]+)", data[x]['title']).group(1))

if data[x]['assignee']['login'] == None:
    print('not assigned yet')
else:
    print(data[x]['assignee']['login'])

print(data[x]['created_at'].replace('T',' ').replace('Z',''))

if data[x]['closed_at'] == None:
    print('Not closed yet')
else:
    print(data[x]['closed_at'].replace('T',' ').replace('Z',''))

In [ ]:
display(Image('https://user-images.githubusercontent.com/68472164/93796393-12fc0e00-fc3b-11ea-8abc-ce360f97083f.png'))

# API EXTRACTION

In [3]:

def getRequests(endpoint, token=os.getenv("apikey"), query_params={}):

    baseUrl = "https://api.github.com"
    url = f"{baseUrl}{endpoint}"

    headers = {
        "Authorization": f"token {token}"
    }

    res = requests.get(url, params=query_params, headers=headers)

    data = res.json()

    if res.status_code != 200:
        raise ValueError(f'Invalid github api call: {data["message"]}')

    return data

In [4]:
sample1 = getRequests("/repos/ironhack-datalabs/datamad0820/pulls",query_params={'state': 'all', 'page' : 1, "per_page":1})

In [5]:
def extractWantedData(data):

    """Create a dictionary with the selected information received from the github api"""
    dictionary=[]
    for x in range(0,len(data)):
        try:
            name={'number':data[x]['number'],
            'title':re.match('\[(.*?)\]',data[x]['title'],re.IGNORECASE).group(1).replace('-',' '),
            'user':data[x]['user']['login'],
            'state':data[x]['state'],
            'created_at':data[x]['created_at'],
            'updated_at':data[x]['updated_at'],
            'closed_at':data[x]['closed_at'],
            'TA':data[x]['assignee']['login'],
            'html_url':data[x]['html_url'],
            'meme':'no'
            }
            dictionary.append(name)
    
        except:
            name={'number':data[x]['number'],
            'title':re.match('\[(.*?)\]',data[x-1]['title'],re.IGNORECASE).group(1).replace('-',' '),
            'user':data[x]['user']['login'],
            'state':data[x]['state'],
            'created_at':data[x]['created_at'],
            'updated_at':data[x]['updated_at'],
            'closed_at':data[x]['closed_at'],
            'TA':'none',
            'html_url':data[x]['html_url'],
            'meme':'no'
            }
            dictionary.append(name)
            
    return dictionary

In [6]:
npages = getRequests("/repos/ironhack-datalabs/datamad0820/pulls",query_params={'state': 'all', 'page' : 1, "per_page":100})[0]['number']

def getDataDict(npages):
    """Get the information from all the github issues pages from a selected repository"""
    pulls = []
    for i in range(1, math.ceil(npages/100) + 1):
        pulls.append(extractWantedData(getRequests("/repos/ironhack-datalabs/datamad0820/pulls",query_params={'state': 'all', 'page' : i, "per_page":100})))
    clean_data = list(itertools.chain.from_iterable(pulls))
    return clean_data


In [7]:
list_pulls = getDataDict(npages)

In [8]:
len(list_pulls)

518

In [9]:
type(list_pulls)

list

In [10]:
type(list_pulls[0])

dict

In [11]:
#Importing tools
#MongoDB
from pymongo import MongoClient

In [12]:
PORT = os.getenv('PORT')
DBURL = os.getenv('DBURL')

In [13]:
def Import_to_mongoDB(lista):
    """Upload the info extractred from github to mongodb"""
    client = MongoClient(port=27017)
    db=client.RankingDB
    for x in range(0,len(lista)):
        result = db.pulls.insert_one(lista[x])
    print(f'finished creating {len(lista)} pull requests')

In [14]:
Import_to_mongoDB(list_pulls)

finished creating 518 pull requests


In [ ]:
PORT


In [ ]:
#sample1

#  WEBSCRAPING

In [15]:
import requests
import bs4
import re
import itertools
import time
import json


In [16]:
url  = 'https://github.com/ironhack-datalabs/datamad0820/network/members'

In [17]:
def getUsers(url):
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    users = re.findall("@\w+\S\w\w*", str(soup))
    users = list(set(users))
    unwanted_users =['@ferrero-felipe', '@WHYTEWYLL', '@agalvezcorell', '@github', '@ironhack-datalabs']
    for i in unwanted_users:
        if i in users:
            users.remove(i)
    dic_users = { i : users[i] for i in range(0, len(users) ) }
    dic_users = {v: k for k, v in dic_users.items()}
    return dic_users

In [18]:
def split_dict(input_dict, chunks=28):
    "Splits dict by keys. Returns a list of dictionaries."
    # prep with empty dicts
    return_list = [dict() for idx in range(chunks)]
    idx = 0
    for k,v in input_dict.items():
        return_list[idx][k] = v
        if idx < chunks-1:  # indexes start at 0
            idx += 1
        else:
            idx = 0
    return return_list

In [19]:
s_users = split_dict(getUsers(url))
s_users

[{'@bmedm': 0},
 {'@Alvere': 1},
 {'@miguelgimenezgimenez': 2},
 {'@Daniel-GarciaGarcia': 3},
 {'@rfminguez': 4},
 {'@RobotKidz': 5},
 {'@MigeruDono': 6},
 {'@AnaMA96': 7},
 {'@marta-zavala': 8},
 {'@Gon41': 9},
 {'@silviaherf': 10},
 {'@gontzalm': 11},
 {'@DiegoCaulonga': 12},
 {'@VanessaMacC': 13},
 {'@IreneLopezLujan': 14},
 {'@jmena23': 15},
 {'@laura290': 16},
 {'@PaulaNuno': 17},
 {'@jorge-alamillos': 18},
 {'@grundius1': 19},
 {'@Diegon8': 20},
 {'@Joycelili': 21},
 {'@CarlosSanzDGP': 22},
 {'@charliesket': 23},
 {'@Davidlazarog': 24},
 {'@FDELTA': 25},
 {'@Jav1-Mart1nez': 26},
 {'@KevsDe': 27}]

In [20]:
def import_users(lista):
    client = MongoClient(port=27017)
    db=client.RankingDB
    for x in range(0,len(lista)):
        result = db.users.insert_one(lista[x])
    print(f'finished adding {len(lista)} users')
    

In [21]:
import_users(s_users)

finished adding 28 users
